Author: Ashkan Nikfarjam
basic info from "https://en.wikipedia.org/wiki/Healthcare_in_Germany"
Germans are offered three mandatory health benefits, which are co-financed by employer and employee: health insurance, accident insurance, and long-term care insurance
list of hospitals in germany: "https://en.wikipedia.org/wiki/List_of_hospitals_in_Germany"

In [26]:
from bs4 import BeautifulSoup
import requests
url = "https://en.wikipedia.org/wiki/List_of_hospitals_in_Germany"
response = requests.get(url)
if response.status_code == 200:
    health_soup = BeautifulSoup(response.content, 'html.parser')
else:
    print('couldnt reach', url)

now we have everything lets write everything in and html file and take a peack

In [27]:
with open("hospitals.html", 'w') as file:
    file.write(health_soup.prettify())

In [28]:
rows = health_soup.find_all('tr')
#now i want to exract all the hospilat names, city and number of beds
# Extract data from each row
hospital_lst = []
city_lst = []
beds_lst = []
for row in rows:
    # Find all table data elements in the row
    cells = row.find_all('td')
    
    # Extract hospital name, city, number of beds
    if len(cells) >= 3:  # Ensure the row has enough data cells
        hospital = cells[0].get_text(strip=True)
        city = cells[1].get_text(strip=True)
        beds = cells[2].get_text(strip=True)
        
        # Remove unwanted characters or tags from beds
        beds = beds.split('[')[0]  # Remove citation references
        
        # store the extracted data
        #print(f"Hospital: {hospital}, City: {city}, Beds: {beds}")
        hospital_lst.append(hospital)
        city_lst.append(city)
        beds_lst.append(beds)

Now that i extracted the data i needed lets create a data frame to represent these

In [29]:
print(len(hospital_lst))
print(len(city_lst))
print(len(beds_lst))

111
111
111


In [43]:
import pandas as pd

health_df = pd.DataFrame({'Hospital Name': hospital_lst, 'City': city_lst, 'Number of beds': beds_lst})
# Remove commas from the 'Number of beds' column and convert to numeric
health_df['Number of beds'] = health_df['Number of beds'].str.replace(',', '').astype(int)

unique_values = health_df['Number of beds'].unique()
print(unique_values)

health_df.head()

[3011 1991 2050 5856 1436 1520 1091 1464 2169 1098  833 1378 1554 1295
 1303 1352 1517 1438 1237 1662 1291 1260 1345  665 1396 1202 1559 1150
  543  656  443  984  919 1457  754  909 1297  673 1428  752 1087  728
  620 1422  569 1057  509  959  761  963  201  236  951 1347  211  573
  138  485 1322  708  632 1699  657  334 1005  574 1300  707  697  450
  525  358  244  300  510  905  499  605  424  655  636  592 1028  782
  562  545  588  619  160 1741  910   24  459   78  240  104  506  367
  307  496   63  433 1281  225  362  195]


,Hospital Name,City,Number of beds
0,Charité – Universitätsmedizin Berlin,Berlin,3011
1,University Hospital Heidelberg,Heidelberg,1991
2,Klinikum der Universität München,Munich,2050
3,Vivantes Hospital Group,Berlin,5856
4,University Medical Center Hamburg-Eppendorf,Hamburg,1436


now we can call this our raw dataframe to use these info for visualization and ML model training one thing could be done is to goup data by city aggregate number of beds and also count the number of hospitals

In [44]:
agg_df = health_df.copy()
#numBeds_hsptl_df = agg_df.groupby(by='City').agg({agg_df.columns[2]:'sum'})
numBeds_hsptl_df = agg_df.groupby('City')['Number of beds'].sum().reset_index()
numBeds_hsptl_df.head()

,City,Number of beds
0,Aachen,1517
1,Amberg,588
2,Augsburg,3440
3,Bad Berka,728
4,Bad Oeynhausen,344


In [45]:
numHsptl = agg_df.City.value_counts()
numHsptl_df = pd.DataFrame(numHsptl).reset_index()
numHsptl_df.head()

,City,count
0,Berlin,9
1,Hamburg,9
2,Kassel,5
3,Munich,5
4,Cologne,4


In [46]:
merged_data = pd.merge(numBeds_hsptl_df, numHsptl_df, on='City')
merged_data.rename(columns={'count':'Number of Hospitals'}, inplace=True)
merged_data.head()

,City,Number of beds,Number of Hospitals
0,Aachen,1517,1
1,Amberg,588,1
2,Augsburg,3440,2
3,Bad Berka,728,1
4,Bad Oeynhausen,344,2


Now i can conver this data frame to csv so i could use the data in my UI file

In [47]:
merged_data.to_csv("numberofHospitals.csv", index=False)

Pharmecies are another set of data to look at here source: https://www.govdata.de/web/guest/suchen/-/details/kreis-viersen-poi-osm-nrw-apotheken-tankstellen77d2f


In [35]:
from googletrans import Translator

pharmacy = pd.read_csv("apotheken_nrw.csv")
pharmacy = pharmacy[['name','adresse']]
pharmacy.adresse = pharmacy.adresse.str.split(',').str[-1]
pharmacy_df = pd.DataFrame(pharmacy.adresse.value_counts()).reset_index()
pharmacy_df.rename(columns={'adresse':'city', 'count':'number of pharmecies'}, inplace=True)
pharmacy_df.head()



ModuleNotFoundError: No module named 'googletrans'

In [ ]:
from fuzzywuzzy import process
german_city = pd.read_csv("../German cities.csv")
#german_city.head()
real_city_lst = german_city.city.to_list()
#real_city_lst
def find_match(city, choices):
    return process.extractOne(city, choices)[0]
test_pd = pharmacy_df.copy()
test_pd['city']= test_pd.city.apply(lambda x: find_match(x, real_city_lst))

test_pd.head()

In [ ]:
test_pd.to_csv("numberOfPharmecies.csv", index=False)